In [1]:
import requests, numpy, time, csv, os
from config import URL, HOST
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm_notebook

HEADERS = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36', 'accept': '*/*'}
FILE = 'products.csv'

def get_html(url, params = None):
    page = requests.get(url, headers = HEADERS, params = params)
    return page

def get_pages_count(html):
    soup = BeautifulSoup(html, 'html.parser')
    pagination = soup.find('div', class_='navigation').find_all('span')[-1].get_text()
    return int(pagination)

def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_ = 'item')
    
    products = []
    for item in items:
        name = item.find('div', class_ = 'name').get_text(strip = True)
        products.append({
            'key_shop': item.find('div', class_ = 'key').get_text().split(' ')[-1],
            'type': item.find('div', class_ = 'type').get_text(strip = True),
            'name': name,
            'description': item.find('div', class_ = 'description').get_text(strip = True),
            'price': int(item.find('div', class_ = 'new').get_text()[:-5].replace(' ','')),
            'link': HOST + item.find('a', title = name).get('href') 
        })
        
    return products

def save_file(items, path):
    with open(path, 'w', newline = '') as file:
        writer = csv.writer(file, delimiter = ';')
        writer.writerow(['Артикул', 'Категория', 'Наименование', 'Описание', 'Цена в руб.', 'Ссылка'])
        for item in items:
            try:
                writer.writerow([item['key_shop'], item['type'], item['name'], item['description'], item['price'], item['link']])
            except UnicodeError:
                writer.writerow([item['key_shop'], item['type'], item['name'], 'Описание уточняйте на сайте', item['price'], item['link']])
            
def parse():
    html = get_html(URL)
    #URL = input('Введите адрес каталога для парсинга: ').strip() or URL
    if html.status_code == 200:
        pages_count = get_pages_count(html.text)
        products =[]
        for page in tqdm_notebook(range(1, pages_count + 1), desc = 'Прогресс парсинга страниц: '):
            html = get_html(URL, params={'list_page': page})
            products.extend(get_content(html.text))
            time.sleep(numpy.random.randint(1,2))
        products = sorted(products, key=lambda x: x['price'])
        
        save_file(products, FILE)
        print(f'Успешно. Получено товаров: {len(products)}.\nФайл {FILE} сохранён в корневой каталог проекта.')
    else:
        print(f'\033[31m Error: {URL} don\'t request. Check your parametrs of \'get_html\' function.')

parse()

Прогресс парсинга страниц:   0%|          | 0/5 [00:00<?, ?it/s]

Успешно. Получено товаров: 95.
Файл products.csv сохранён в корневой каталог проекта.
